In [1]:
import matplotlib.pyplot as plt
import numpy as np

def hinton(matrix, max_weight=None, ax=None):
    """Draw Hinton diagram for visualizing a weight matrix."""
    ax = ax if ax is not None else plt.gca()

    if not max_weight:
        max_weight = 2 ** np.ceil(np.log(np.abs(matrix).max()) / np.log(2))

    ax.patch.set_facecolor('gray')
    ax.set_aspect('equal', 'box')
    ax.xaxis.set_major_locator(plt.NullLocator())
    ax.yaxis.set_major_locator(plt.NullLocator())

    for (x, y), w in np.ndenumerate(matrix):
        color = 'white' if w > 0 else 'black'
        size = np.sqrt(np.abs(w) / max_weight)
        rect = plt.Rectangle([x - size / 2, y - size / 2], size, size, facecolor=color, edgecolor=color)
        ax.add_patch(rect)

    ax.autoscale_view()
    ax.invert_yaxis()

In [2]:
def pearson(x, y):
    temp_x = np.array(x)
    temp_y = np.array(y)
    notnullx = np.where(temp_x != 0)[0]
    notnully = np.where(temp_y != 0)[0]
    indices = list(set(notnullx) & set(notnully))
    p = pearsonr(temp_x[indices], temp_y[indices])[0]
    return p if p == p else 0

# Start - CF

In [3]:
import pandas as pd
from scipy.spatial.distance import cosine
from scipy.stats import pearsonr